In [2]:
!pip install streamlit -q
     
     

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 KB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.4 MB/s eta 0:00:00


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.5 MB/s eta 0:00:00


In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/medquad_model')
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/medquad_model')


In [23]:
%%writefile app.py
import streamlit as st
import torch
import numpy as np
a = []
per_value = []
perplexity=[]
result=[]
st.title("AI Medical Assistant")

# Get user input
st.header('Some examples')
st.write('At what age psoriasis occurs')
st.write('Symptoms of asthma')
st.write('What causes lung cancer')
text = st.text_input("Please Provide your text:")


from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/new_model')
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/new_model')

# Generate text

if len(text)>0:
  input_ids = tokenizer.encode(text, return_tensors='pt')
  sample_outputs = model.generate(
                                  input_ids, 
                                  #bos_token_id=random.randint(1,30000),
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 100,
                                  top_p=0.95, 
                                  num_return_sequences=3,
                                  temperature=1.0
                                  )

  st.write("Generated Sentences:")
  for i, sample_output in enumerate(sample_outputs):
    a.append(sample_output)
    generated_text = tokenizer.decode(sample_output, skip_special_tokens=True)
    answer = generated_text.replace(text, '').strip()
    result.append(answer)
    st.write(generated_text)
  
# givining the values to the perplexity metrix
  query = text
  generated_sentences = result
  # Calculate perplexity scores for each generated sentence
  perplexities = []
  for sentence in generated_sentences:
      input_ids = tokenizer.encode(query + " " + sentence, return_tensors="pt")
      with torch.no_grad():
          logits = model(input_ids)[0]
      perplexity = np.exp(torch.mean(torch.nn.functional.cross_entropy(
          logits.view(-1, logits.size(-1)), input_ids.view(-1)
      ), dim=0))
      perplexities.append(perplexity.item())

  # Select the sentence with the lowest perplexity
  best_sentence_index = perplexities.index(min(perplexities))
  best_sentence = generated_sentences[best_sentence_index]
  st.header('perplexity 1(Updated)')
  st.write(best_sentence)

  

  # by using the previous
  perplexity=[]
  for x in a:
    input_ids = x
    # print(sample_output)

    # Calculate the perplexity of the generated text
    loss = model(input_ids, labels=input_ids).loss
    perplex = torch.exp(loss)
    perplexity.append(perplex.item())
  st.header('perplexity 2 (OLD)')
  best_sentence_index = perplexity.index(min(perplexity))
  st.write(result[best_sentence_index])

else:
  st.write('Welcome to GPT2')
# Create a "Regenerate" button


# Display output



Overwriting app.py


In [6]:
# import streamlit as st

# # Define the model and tokenizer
# model = ...
# tokenizer = ...

# # Define a function to generate text
# def generate_text(text):
#   if len(text)>0:
#     input_ids = tokenizer.encode(text, return_tensors='pt')
#     sample_outputs = model.generate(
#                                     input_ids, 
#                                     do_sample=True,   
#                                     top_k=50, 
#                                     max_length = 100,
#                                     top_p=0.95, 
#                                     num_return_sequences=3,
#                                     temperature=1.0
#                                     )

#     # Display the generated text
#     st.write("Generated Sentences:")
#     for i, sample_output in enumerate(sample_outputs):
#       st.write(f"{i+1}. {tokenizer.decode(sample_output, skip_special_tokens=True)}")

# # Create a text input box
# text = st.text_input("Enter some text:")

# # Generate the initial text
# generate_text(text)

# # Create a "Regenerate" button
# if st.button("Regenerate"):
#   generate_text(text)


In [7]:

!pip install pyngrok
     

from pyngrok import ngrok
     


     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19790 sha256=97c8d8b9eae6e2ddd80ee2318b815e728da58db2abb2c740c74ae03b4f456316
  Stored in directory: /root/.cache/pip/wheels/f6/89/59/49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


In [8]:
ngrok.set_auth_token("2LoeoedCqusM7VxjDHJfm1b9TJg_5wRMcFESTSHY6u26tUoMj") #ngrok.com
     



In [9]:
!killall ngrok

ngrok: no process found


In [10]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://4204-34-125-107-7.ngrok.io" -> "http://localhost:80"


In [ ]:

from pyngrok import ngrok

tunnels = ngrok.get_tunnels()
tunnels

INFO:pyngrok.process.ngrok:t=2023-04-03T07:06:56+0000 lvl=info msg=start pg=/api/tunnels id=8bf472c60f2792dc
2023-04-03 07:06:56.645 t=2023-04-03T07:06:56+0000 lvl=info msg=start pg=/api/tunnels id=8bf472c60f2792dc


[<NgrokTunnel: "https://ba36-35-199-167-246.ngrok.io" -> "http://localhost:80">,
 <NgrokTunnel: "http://ba36-35-199-167-246.ngrok.io" -> "http://localhost:80">]

INFO:pyngrok.process.ngrok:t=2023-04-03T07:06:56+0000 lvl=info msg=end pg=/api/tunnels id=8bf472c60f2792dc status=200 dur=299.829µs
2023-04-03 07:06:56.654 t=2023-04-03T07:06:56+0000 lvl=info msg=end pg=/api/tunnels id=8bf472c60f2792dc status=200 dur=299.829µs


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.6 MB/s eta 0:00:00


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/medquad_model')
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/medquad_model')


In [ ]:



model.eval()
result=[]

# Prepare the input text
input_text = "how to prevent from heart attack"

# Encode the input text using the tokenizer
input_ids = tokenizer.encode(input_text, return_tensors='pt')
input_ids

# Generate the output sequence
output = model.generate(input_ids,
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 100,
                                    top_p=0.95,
                                    temperature=1,                         
                                    num_return_sequences=3)
# Decode the output sequence
# output = model.generate(input_ids, max_length=1000, do_sample=True)
a = []
for i, sample_output in enumerate(output):
  a.append(sample_output)
  generated_text = tokenizer.decode(sample_output, skip_special_tokens=True)
  answer = generated_text.replace(input_text, '').strip()
  result.append(answer)




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:

result

['and stroke. To prevent heart attack and stroke your doctor may need to treat some of the following conditions Heart attacks and stroke Your doctor may prescribe medicines or devices to control a heart attack or stroke. Medicines or devices can keep the heart beating longer and help control blood pressure and blood pressure. Examples of medicines for heart attacks and stroke include aspirin acetaminophen ibuprofen and naproxen. If you have any of these conditions or conditions take any of these medicines',
 '. - You can take steps to help prevent heart attack. - Know the signs and symptoms of heart attack including how serious it is and how quickly they can worsen. Keep a copy of each health care card a record of your heart attack symptoms. - Take an aspirin every day. People with heart attacks should get one to two aspirin breaks each day. - If you have symptoms of heart attack talk to your doctor and make sure their doctor knows them. - If an attack',
 'and stroke. Heart attack and 

In [ ]:
import torch
per_value = []
for x in a:
  input_ids = x
  # print(sample_output)


  # Calculate the perplexity of the generated text
  loss = model(input_ids, labels=input_ids).loss
  perplexity = torch.exp(loss)
  per_value.append(perplexity.item())

In [ ]:
per_value

[4.5889973640441895, 5.0325093269348145, 4.5058746337890625]

In [ ]:
import json
import requests

API_URL = "https://api-inference.huggingface.co/models/sentence-transformers/msmarco-distilbert-base-tas-b"
headers = {"Authorization": f"Bearer {api_token}"}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

data = query(
    {
        "inputs": {
            "source_sentence": "That is a happy person",
            "sentences": [
                "That is a happy dog",
                "That is a very happy person",
                "Today is a sunny day"
            ]
        }
    }
)
##[0.853, 0.981, 0.655]

NameError: ignored

In [ ]:
best_sentence

NameError: ignored

In [ ]:
!pip3 install biopython



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.2 MB/s eta 0:00:00


In [ ]:
import en_core_web_sm

nlp = en_core_web_sm.load()

In [ ]:
from Bio import Entrez, Medline, BiopythonWarning
import warnings
import spacy

# Set up a spacy model for medical domain
# nlp = spacy.load('en_core_medical_sm')

# Set up the BioPython library for accessing medical ontology
Entrez.email = "Your.Name.Here@example.org"
warnings.filterwarnings("ignore", category=BiopythonWarning)

# Define a function to calculate the relevance score of a sentence
def calculate_relevance(sentence):
    # Parse the sentence using the spacy model
    doc = nlp(sentence)

    # Retrieve the concepts from the ontology that are relevant to the parsed sentence
    relevant_concepts = []
    for token in doc:
        query = token.text.lower()
        handle = Entrez.esearch(db="mesh", term=query)
        record = Entrez.read(handle)
        if record["Count"] != "0":
            relevant_concepts.append(query)

    # Calculate the relevance score based on the number of relevant concepts found
    relevance_score = len(relevant_concepts) / len(doc)

    return relevance_score

# Example sentence to evaluate
generated_sentence = "The patient has a fever and headache."

# Calculate the relevance score of the generated sentence
relevance_score = calculate_relevance(generated_sentence)

print(f"Relevance score: {relevance_score}")


URLError: ignored